<p style="text-align: center">
    <b><span style="float: center; font-size: 12pt">LocalThingsNetwork</span>
    <br>
    <span style="font-size: 40pt"> Layered network demonstration </span></b>
</p>

---

This is a demonstration of the `localthingsnet.network.MainServer` and `localthingsnet.network.SubServer` classes used to build multi-level hierarchical network structures. Both classes extend the `Server` class. The ***MainServer*** acts as the central *(root)* node of a network. It controls and manages all other server nodes in the network. The *SubServer* is a combination of the *Server* and *Client* class which means client can connect to it but it also can connect to another (sub-)server. This demonstration shows how to to **initialize** the different server and how to **connect** them to a multi layered network.

Please work through the *Sever* and *Client* demonstration beforehand. 

> ***NOTE:** The Mainserver is currently just a PoC and does not provide much functionality*

---

In [1]:
import localthingsnet.network as ltn

---

# ***Main-/Subserver initialization***

The initialization for Main- and SubServer are similar to the `Server` initialization. Likewise `startServer()` needs to be called to bind the server to an address.

>***NOTE:** If port 4000 is not free, change the preferred port of the mainserver to another port*

In [2]:
main = ltn.MainServer(preferredport=4000)

sub1 = ltn.SubServer(servername='sub1')
sub2 = ltn.SubServer(servername='sub2')
sub3 = ltn.SubServer(servername='lower_sub3')

In [3]:
main.startServer()

Serversock bound to 4000
Server is running at ('192.168.178.140', 4000)
Maindata Datasock bound to 3999
Mainserver operational
Started services


In [4]:
sub1.startServer()

Serversock bound to 4001
Server is running at ('192.168.178.140', 4001)
Subserver 'sub1' operational
Started services


In [5]:
sub2.startServer()

Serversock bound to 4002
Server is running at ('192.168.178.140', 4002)
Subserver 'sub2' operational
Started services


In [6]:
sub3.startServer()

Serversock bound to 4003
Server is running at ('192.168.178.140', 4003)
Subserver 'lower_sub3' operational
Started services


---

# ***Build multi layered network***

Since the subserver is an combination of server and client it can use the client `connect()` function to connect to another server, subserver or mainserver. The server instance a subserver connects to is referrd to as *higher server*. The subserver can disconnect or connect to another higher server at any time.

In [7]:
sub1.connect(main.addr)

Connecting to 192.168.178.140
Connected with 192.168.178.140 at 4000
Connected new datasocket to port 3999
192.168.178.140 is registered as 'sub1'
[13:57:33]Mainserver> Connected to Mainserver


'Connected'

[13:57:33]Mainserver> Registered as 'sub1'
[13:57:33]Mainserver> Type 's.help()' for more information
[13:57:33]Mainserver> You have admin permissions


In [8]:
sub2.connect(main.addr)

Connecting to 192.168.178.140
Connected with 192.168.178.140 at 4000
Connected new datasocket to port 3999


'Connected'

192.168.178.140 is registered as 'sub2'
[13:57:34]Mainserver> Connected to Mainserver
[13:57:34]Mainserver> Registered as 'sub2'
[13:57:34]Mainserver> Type 's.help()' for more information
[13:57:34]Mainserver> You have admin permissions


In [9]:
sub3.connect(sub1.addr)

Connecting to 192.168.178.140
Connected with 192.168.178.140 at 4001


'Connected'

192.168.178.140 is registered as 'lower_sub3'
[13:57:35]sub1> Connected to sub1
[13:57:35]sub1> Registered as 'lower_sub3'
[13:57:35]sub1> Type 's.help()' for more information
[13:57:35]sub1> You have admin permissions


In [10]:
# Connect some clients
c1 = ltn.Client('client1')
c1.connect(main.addr)

Connecting to 192.168.178.140
Connected with 192.168.178.140 at 4000
Connected new datasocket to port 3999
192.168.178.140 is registered as 'client1'


'Connected'

Connected to Mainserver
Registered as 'client1'
Type 's.help()' for more information
You have admin permissions


In [11]:
c2 = ltn.Client('client2')
c2.connect(main.addr)

Connecting to 192.168.178.140
Connected with 192.168.178.140 at 4000
192.168.178.140 is registered as 'client2'
Connected new datasocket to port 3999
Connected to Mainserver
Registered as 'client2'
Type 's.help()' for more information
You have admin permissions


'Connected'

In [12]:
c3 = ltn.Client('client3')
c3.connect(sub1.addr)

Connecting to 192.168.178.140
Connected with 192.168.178.140 at 4001


'Connected'

192.168.178.140 is registered as 'client3'
Connected to sub1
Registered as 'client3'
Type 's.help()' for more information
You have admin permissions


In [13]:
c4 = ltn.Client('client4')
c4.connect(sub3.addr)

Connecting to 192.168.178.140
Connected with 192.168.178.140 at 4003


'Connected'

192.168.178.140 is registered as 'client4'
Connected to lower_sub3
Registered as 'client4'
Type 's.help()' for more information
You have admin permissions


---

# ***Interact with (higher) server***

A subserver can interact with its higher server through `sendMsg()`, `sendData()`, `sendRequest()` and `sendCommand()`. These are the functions implemented in the `Client` class. A subserver can send something to a connected client *(or subserver)* through `sendMsgTo()`, `sendDataTo()`, `sendRequestTo()`, `sendCommandTo()`. These are the functions implemented in the `Server` class. In general use the '...To()' functions to send to a client *(lower connection)* and the functions without To' when sending to a sever *(higher connection)*.

Subserver have additional servercommands and requestables. Two important servercommands are `'s.sethigherserver()'` and `'s.connecttoghigher()'`. These allow to set the higher server of a subserver through a client *(or lower subserver)* if the client has admin permissions. The `'s.connecttohigher()'` allows a client to connect to the higher server. For subserver this means they change their higher server to the higher server of the connected higher server, effectively moving up a layer in the network structure. The mainserver has the `'s.listnetwork()'` servercommand to display the whole network structure with every subserver and client.

> ***NOTE:** THE `'s.listnetwork()'` servercommand may be formatted wrong in VS Code*

In [14]:
c4.sendMsg(f's.sethigherserver{sub3.addr}')

[13:57:41]client4: s.sethigherserver('192.168.178.140', 4003)
Confirm connecting to a new address by sending 's.sethigherserver('192.168.178.140', 4003)'


In [15]:
c4.sendMsg(f's.sethigherserver{sub3.addr}')

[13:57:42]client4: s.sethigherserver('192.168.178.140', 4003)
Disconnecting from higher Server
Disconnected from Server 'sub1'
Lost connection to socket at 4001
Connecting to '192.168.178.140'
Disconnected connection 'lower_sub3'
Lost connection with higher Server
No server found
Disconnected from Server
Connect result: No Server on this address


In [16]:
c1.sendMsg('s.listnetwork()')

[13:57:43]client1: s.listnetwork()
All connected instances in the network:

Mainserver(192.168.178.140)  2 servers  2 clients
 ├──> client1(192.168.178.140)
 ├──> client2(192.168.178.140)
 |
 ├─ sub1(192.168.178.140)  0 servers  1 clients
 |  ├──> client3(192.168.178.140)
 | 
 ├─ sub2(192.168.178.140)  0 servers  0 clients
   

---

# ***Network shutdown***

Subserver and mainserver can be shut down with `shutdownServer()`. Subnets for a main/-subserver will not stop operating if their higher server shuts down. The only exception is the mainserver servercommand `'s.shutdownnetwork()'` which will disable every subserver in the network. If using the co

The mainserver also has servercommands to restart or shutdown certain server.

In [17]:
# Single server shutdown
sub3.shutdownServer()

Disconnecting from higher server
Disconnected from Server
Shutting Server down
Stopping services
Disconnecting all users
Disconnected connection 'client4'
Unbinding all sockets
Closing datasock 'Serversock' at 4003
Server is inactive
Lost connection to socket at 4003
Disconnected from Server 'lower_sub3'


Ended services


In [18]:
# Whole network shutdown
c1.sendMsg('s.shutdownnetwork()')

[13:57:46]client1: s.shutdownnetwork()


Send 's.shutdownnetwork()' again to confirm the shutdown of the whole network


In [19]:
# Confirmation
c1.sendMsg('s.shutdownnetwork()')

[13:57:47]client1: s.shutdownnetwork()
[13:57:47]Mainserver> Network is shutting down
[13:57:47]Mainserver> Network is shutting down
Disconnecting from higher server
Disconnecting from higher Server
Disconnecting from higher server
Disconnecting from higher Server


Disconnected from Server 'Mainserver'Lost connection to socket at 3999

Shutting Server down
Stopping services
Disconnecting all users
Lost connection to socket at 3999
Lost connection to socket at 4001
Disconnected connection 'client3'
Disconnected from Server 'sub1'
Disconnected from Server 'Mainserver'
Shutting Server down
Stopping services
Disconnecting all users
Unbinding all sockets
Network is shutting down
WAR: Senderror for a disconnected client
Closing datasock 'Serversock' at 4002
Unbinding all sockets
Lost connection to socket at 4000
Network is shutting down
Shutting Server down
Stopping services
Disconnecting all users
Closing datasock 'Serversock' at 4001
Server is inactive
Lost connection to socket at 4000
Disconnected from Server 'Mainserver'
Lost connection to socket at 3999
Disconnected connection 'sub1'
Disconnected connection 'sub2'
Lost connection to socket at 4000
Disconnected from Server 'Mainserver'
Lost connection to socket at 3999
Server is inactive
Lost conne

---
<span style="float: right; font-size: 15pt"><b>LocalThingsNetwork</b></i></span>
<br><br>
<span style="float: right; font-size: 10pt"><i>For MainServer version </i><b>2.09.040</b><br>
<span style="float: right; font-size: 10pt"><i>For SubServer version </i><b>4.17.075</b><br>